<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split

# **CallBack**

In [2]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.70):
            print("\nReached 70.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [2]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

'git' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
train_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\train"
test_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\test"

In [4]:
assert len(os.listdir(train_dir)) == 23 and len(os.listdir(test_dir)) == 23

# **Image Generator**

In [5]:
data_generator = ImageDataGenerator(validation_split = 0.2)

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=40,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
                                  horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)
#                                   rotation_range=40
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True)
                                 

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(200,200))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(200,200))


Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


## Model_1

In [16]:
i = Input(shape = (200,200,3))
x = Conv2D(16,(3,3),activation='relu')(i)
x = MaxPooling2D((2,2))(x)
x = Conv2D(32,(3,3),activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64,(3,3),activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128,(3,3),activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(512,activation='relu',kernel_regularizer = l2())(x)
# x = Dropout(0.2)(x)
x = Dense(23,activation='softmax')(x)

In [17]:
model = Model(i,x)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 23, 23, 64)        0   

In [18]:
model.compile(optimizer = Adam(),
          loss = 'sparse_categorical_crossentropy',
          metrics = ['sparse_categorical_accuracy'])

In [19]:
history = model.fit_generator(train_generator,validation_data=validation_generator,
                              epochs=30,steps_per_epoch = train_generator.n//train_generator.batch_size,
                              validation_steps = validation_generator.n//validation_generator.batch_size)

Epoch 1/30
243/243 [==============================] - 1294s 5s/step - loss: 3.3249 - sparse_categorical_accuracy: 0.1275 - val_loss: 2.9600 - val_sparse_categorical_accuracy: 0.1386
Epoch 2/30
243/243 [==============================] - 1082s 4s/step - loss: 2.8901 - sparse_categorical_accuracy: 0.1646 - val_loss: 2.8736 - val_sparse_categorical_accuracy: 0.1582
Epoch 3/30
243/243 [==============================] - 719s 3s/step - loss: 2.8473 - sparse_categorical_accuracy: 0.1770 - val_loss: 2.8458 - val_sparse_categorical_accuracy: 0.1681
Epoch 4/30
243/243 [==============================] - 732s 3s/step - loss: 2.8219 - sparse_categorical_accuracy: 0.1956 - val_loss: 2.8636 - val_sparse_categorical_accuracy: 0.1953
Epoch 5/30
243/243 [==============================] - 903s 4s/step - loss: 2.7963 - sparse_categorical_accuracy: 0.2095 - val_loss: 2.8185 - val_sparse_categorical_accuracy: 0.1943
Epoch 6/30
243/243 [==============================] - 668s 3s/step - loss: 2.7793 - sparse_ca

KeyboardInterrupt: 